In [2]:
import pandas as pd
import spacy
nlp = spacy.load("en_core_web_trf")

In [2]:
df = pd.read_csv('meta.csv', index_col=0)
type_summary = df.groupby('review_type').agg({'review_type':['count',]}).reset_index(drop=False).sort_values(by=('review_type', 'count'), ascending=False)
type_summary

review_type      
                count
5  single_focus  1093
2         multi   487
4    not_review   247
1       cluster   244
0         brief    82
3        no_pdf     2

In [3]:
df_single = pd.read_csv('single_author_meta.csv', index_col=0)
df_single_genre_summary = df_single.groupby('genre_parsed').agg({'genre_parsed':['count',]}).reset_index(drop=False).sort_values(by=('genre_parsed', 'count'), ascending=False)
df_single_genre_summary

genre_parsed      
               count
2   nonfiction   801
1      fiction   226
3       poetry    34
0        drama    18

In [5]:
reviews_all = []
for i in df_single['record_id']:
    with open (f'txt/{i}.txt') as f:
        this_review = f.read()
    f.close()
    reviews_all.append(this_review)
len(reviews_all)

1079

In [143]:
%%time

spacy_docs_all = []

for doc in nlp.pipe(reviews_all, disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"]):
    spacy_docs_all.append(doc)

CPU times: user 6h 14min 50s, sys: 49min 3s, total: 7h 3min 54s
Wall time: 2h 3min 58s


In [144]:
ents_all = []
# preprocess entities
for e, s in enumerate(spacy_docs_all):
    ents = []
    for ent in s.ents:
        ents.append([e, ent.text, ent.start_char, ent.end_char, ent.label_])    
    ents_all.extend(ents)
len(ents_all)

46891

In [149]:
single_ids = [[e, i] for e,i in enumerate(df_single['record_id'])]
df_single_ids = pd.DataFrame.from_records(df_single_ids, columns=['e', 'record_id']).set_index('e')
df_single_ids 

,record_id
e,
0,124900101
1,89760874
2,89659668
3,124736362
4,125250815
...,...
1074,89997087
1075,89977217
1076,89967886


In [156]:
df_ents = pd.DataFrame.from_records(ents_all, columns=['e', 'text', 'start_char', 'end_char', 'label']).set_index('e')
df_ents_joined = df_ents.join(df_single_ids ).set_index('record_id').reset_index(drop=False)
df_ents_joined.to_csv('extracted_features/spacy_entities_all.csv')